### Determinando un FVG
Lo primero que debemos hacer es leer data.
Vamos a descargar data de BTCUSDT en el rango 2023-08-09 - 2023-08-11 (9 al 11 de Agosto de 2023) en la temporalidad M5

In [21]:
import pandas as pd
from lightweight_charts import Chart
from finta import TA
import zipfile
import os

Este es un zip que tiene los datos de un solo dia 11 de Agosto en M5

In [22]:
zipfile_name = 'BTCUSDT-5m-2023-08-11.zip'
file_name = 'BTCUSDT-5m-2023-08-11.csv'
data_path = os.path.join('btc_fvg', zipfile_name)
# btc_fvg\BTCUSDT-5m-2023-08-11.zip

In [23]:
def read_csv(data_path, file_name):
    """
    Function to read csv inside zip.
    Input path strings
    Output: Pandas Dataframe
    """ 
    with zipfile.ZipFile(data_path, 'r') as zip_ref:
        with zip_ref.open(file_name) as csv_file:
            # Lee el archivo CSV utilizando Pandas
            dataframe = pd.read_csv(csv_file)
            return dataframe


In [24]:
data = read_csv(data_path, file_name)
data.head()

,open_time,open,high,low,close,volume,close_time,quote_volume,count,taker_buy_volume,taker_buy_quote_volume,ignore
0,1691712000000,29443.7,29443.8,29439.5,29439.5,142.335,1691712299999,4.190503e+06,1814,33.670,9.912864e+05,0
1,1691712300000,29439.6,29442.9,29436.0,29442.8,222.348,1691712599999,6.545603e+06,1957,149.725,4.407645e+06,0
2,1691712600000,29442.8,29450.3,29442.8,29449.0,314.360,1691712899999,9.257319e+06,2312,210.901,6.210576e+06,0
3,1691712900000,29449.0,29459.9,29445.5,29459.8,400.268,1691713199999,1.178984e+07,3253,244.879,7.212872e+06,0
4,1691713200000,29459.9,29475.0,29459.9,29475.0,397.210,1691713499999,1.170583e+07,3365,261.123,7.695420e+06,0


In [25]:
# Function to clean data to what i need

def prep_data(dataframe):
    # Eliminating irrelevant columns
    dataframe = dataframe[['open_time', 'open', 'high', 'low','close', 'volume']]
    dataframe['open_time'] = pd.to_datetime(dataframe['open_time'],unit='ms')
    return dataframe

In [26]:
aa = prep_data(data)

C:\Users\eduar\AppData\Local\Temp\ipykernel_9404\3479782272.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataframe['open_time'] = pd.to_datetime(dataframe['open_time'],unit='ms')


In [27]:
aa

,open_time,open,high,low,close,volume
0,2023-08-11 00:00:00,29443.7,29443.8,29439.5,29439.5,142.335
1,2023-08-11 00:05:00,29439.6,29442.9,29436.0,29442.8,222.348
2,2023-08-11 00:10:00,29442.8,29450.3,29442.8,29449.0,314.360
3,2023-08-11 00:15:00,29449.0,29459.9,29445.5,29459.8,400.268
4,2023-08-11 00:20:00,29459.9,29475.0,29459.9,29475.0,397.210
...,...,...,...,...,...,...
283,2023-08-11 23:35:00,29416.1,29416.9,29413.4,29413.4,176.268
284,2023-08-11 23:40:00,29413.5,29420.0,29413.4,29419.9,139.629
285,2023-08-11 23:45:00,29420.0,29430.0,29412.2,29412.2,311.631
286,2023-08-11 23:50:00,29412.3,29416.0,29412.2,29415.9,88.854


In [28]:
# Data for chart

bb = aa.copy()
bb = bb.rename(columns={"open_time": "time"})
bb

,time,open,high,low,close,volume
0,2023-08-11 00:00:00,29443.7,29443.8,29439.5,29439.5,142.335
1,2023-08-11 00:05:00,29439.6,29442.9,29436.0,29442.8,222.348
2,2023-08-11 00:10:00,29442.8,29450.3,29442.8,29449.0,314.360
3,2023-08-11 00:15:00,29449.0,29459.9,29445.5,29459.8,400.268
4,2023-08-11 00:20:00,29459.9,29475.0,29459.9,29475.0,397.210
...,...,...,...,...,...,...
283,2023-08-11 23:35:00,29416.1,29416.9,29413.4,29413.4,176.268
284,2023-08-11 23:40:00,29413.5,29420.0,29413.4,29419.9,139.629
285,2023-08-11 23:45:00,29420.0,29430.0,29412.2,29412.2,311.631
286,2023-08-11 23:50:00,29412.3,29416.0,29412.2,29415.9,88.854


### FVG Logic
Partamos de lo siguiente:
- Un FVG se forma con 3 velas ya cerradas, partiendo de la vela actual debemos ver dos velas atrás.
- Un FVG se calcula cuando la vela ya cerró.
- Cada vela que se crea se debe revisar nuevamente si ha creado un FVG con las dos velas anteriores.
- Se debe iniciar el analisis en una vela que sea la tercera, para que tenga dos velas anteriores.

##### Bullish FVG
Logica FVG:
Tenemos vela 1,2 y 3

- Si el low de vela 3 es mayor que el high de vela 1 Y ADEMAS el low de la vela 3 es mayor que el high  de la vela 2

    Price.Low (last) is greater than Price.High (2 candles ago) – This defines the lack of overlap between candle wicks
    Price.Close (1 candle ago) is greater than Price.Open (1 candle ago) by at least ‘x %’ – This defines the large buying pressure
    Price.Close (1 candle ago) is greater than Price.High  (2 candles ago) – This defines that the candle with the buying pressure closed above the high of the previous candle

- Definimos un gap minimo, porcentual. GAP = %
- EL LOW de la vela actual (last) es *mayor* que el HIGH de la vela 2 posiciones anterior.
- El CLOSE de la vela anterior (1 posicion anterior) es *mayor* que el OPEN de la vela 1 posicion anterior por al menos el GAP porcentual.
- El CLOSE de la vela 1 posicion anterior es *mayor* que el HIGH de la vela 2 posisciones anterior.


##### Bearish FVG
    Price.High (last) is less than Price.Low (2 candles ago) – This defines the lack of overlap between candle wicks
    Price.Close (1 candle ago) is less than Price.Open (1 candle ago) by at least ‘x %’ – This defines the large selling pressure
    Price.Close (1 candle ago) is less than Price.Low (2 candles ago) – This defines that the candle with the selling pressure closed below the low of the previous candle

- Definimos un gap minimo, porcentual. GAP = %
- El HIGH de la vela actual (last) es *menor* que el 

In [29]:
# Setear el index como el timestamp de la
bb.set_index('time', inplace=True)

In [38]:
fila_actual = bb.iloc[3-1]

len(bb)-1

287

In [45]:
# funcion para determinar FVG. SE dejará para después.

def bullish_fvg_detector(dataframe):
    # Define porcentual GAP
    
    # Obtain df lenght
    lenght = len(dataframe)
    
    # Entramos por cada fila
    for candle in range(lenght):
        #print(f"La vela {candle} tiene como open price {dataframe.iloc[candle]['open']}")
        if candle >= 2:
            # Bulish FVG
            if dataframe.iloc[candle]['low'] > dataframe.iloc[candle-2]['high']\
            and dataframe.iloc[candle-1]['close'] > dataframe.iloc[candle-1]['open']\
            and dataframe.iloc[candle-1]['close'] > dataframe.iloc[candle-2]['high']:
                print(f"La vela {candle} tiene un FVG")
            
            # Bearish FVG:
            if 

            
    return
    



bullish_fvg_detector(bb)

La vela 3 tiene un FVG
La vela 4 tiene un FVG
La vela 5 tiene un FVG
La vela 9 tiene un FVG
La vela 15 tiene un FVG
La vela 22 tiene un FVG
La vela 37 tiene un FVG
La vela 38 tiene un FVG
La vela 42 tiene un FVG
La vela 48 tiene un FVG
La vela 49 tiene un FVG
La vela 54 tiene un FVG
La vela 64 tiene un FVG
La vela 65 tiene un FVG
La vela 67 tiene un FVG
La vela 68 tiene un FVG
La vela 71 tiene un FVG
La vela 82 tiene un FVG
La vela 83 tiene un FVG
La vela 97 tiene un FVG
La vela 98 tiene un FVG
La vela 99 tiene un FVG
La vela 117 tiene un FVG
La vela 123 tiene un FVG
La vela 124 tiene un FVG
La vela 133 tiene un FVG
La vela 138 tiene un FVG
La vela 139 tiene un FVG
La vela 142 tiene un FVG
La vela 156 tiene un FVG
La vela 157 tiene un FVG
La vela 158 tiene un FVG
La vela 177 tiene un FVG
La vela 192 tiene un FVG
La vela 208 tiene un FVG
La vela 217 tiene un FVG
La vela 225 tiene un FVG
La vela 226 tiene un FVG
La vela 232 tiene un FVG
La vela 233 tiene un FVG
La vela 235 tiene un FVG
L

In [32]:
bb

,open,high,low,close,volume
time,,,,,
2023-08-11 00:00:00,29443.7,29443.8,29439.5,29439.5,142.335
2023-08-11 00:05:00,29439.6,29442.9,29436.0,29442.8,222.348
2023-08-11 00:10:00,29442.8,29450.3,29442.8,29449.0,314.360
2023-08-11 00:15:00,29449.0,29459.9,29445.5,29459.8,400.268
2023-08-11 00:20:00,29459.9,29475.0,29459.9,29475.0,397.210
...,...,...,...,...,...
2023-08-11 23:35:00,29416.1,29416.9,29413.4,29413.4,176.268
2023-08-11 23:40:00,29413.5,29420.0,29413.4,29419.9,139.629
2023-08-11 23:45:00,29420.0,29430.0,29412.2,29412.2,311.631


##### Bearish FVG
- Si el high de la vela 3 es menor que el low de la vela 1 Y ADEMAS el high de la vela 3 es menor que el high de la vela 2